# Evaluating Bot Performance (Chess)

In [51]:
import os

parent = '/'.join(s for s in os.getcwd().split('\\')[:-1])
print(parent)

C:/Users/Ben/Downloads/chappie.ai-test-ben


In [102]:
import pandas as pd

ai = 'test'

game_df = pd.read_csv(f'{parent}/skills/chess/data/models/{ai}/logs/game_log.csv')
game_df.head(10)

,state0,state1,state2,state3,state4,state5,state6,state7,state8,state9,...,prob4092,prob4093,prob4094,prob4095,action,value,reward,ELO,Game-ID,Date
0,0.0,12.0,10.0,11.0,13.0,14.0,11.0,10.0,12.0,9.0,...,0.0,0.0,0.0,0.0,3307.0,1.0,0.0,NaN,SRFXAXNBRFQN30S,2021-12-17 13:25:57.623929
1,1.0,12.0,10.0,11.0,13.0,14.0,11.0,10.0,12.0,9.0,...,0.0,0.0,0.0,0.0,666.0,-1.0,0.0,NaN,SRFXAXNBRFQN30S,2021-12-17 13:25:57.623929
2,0.0,12.0,10.0,11.0,13.0,14.0,11.0,10.0,12.0,9.0,...,0.0,0.0,0.0,0.0,3364.0,1.0,0.0,NaN,SRFXAXNBRFQN30S,2021-12-17 13:25:57.623929
3,1.0,12.0,10.0,11.0,13.0,14.0,11.0,10.0,12.0,9.0,...,0.0,0.0,0.0,0.0,536.0,-1.0,0.0,NaN,SRFXAXNBRFQN30S,2021-12-17 13:25:57.623929
4,0.0,12.0,10.0,11.0,13.0,14.0,11.0,10.0,12.0,2.0,...,0.0,0.0,0.0,0.0,3956.0,1.0,0.0,NaN,SRFXAXNBRFQN30S,2021-12-17 13:25:57.623929
5,1.0,12.0,10.0,11.0,13.0,14.0,11.0,10.0,12.0,2.0,...,0.0,0.0,0.0,0.0,80.0,-1.0,0.0,NaN,SRFXAXNBRFQN30S,2021-12-17 13:25:57.623929
6,0.0,12.0,2.0,11.0,13.0,14.0,11.0,10.0,12.0,2.0,...,0.0,0.0,0.0,0.0,3756.0,1.0,0.0,NaN,SRFXAXNBRFQN30S,2021-12-17 13:25:57.623929
7,1.0,12.0,2.0,11.0,13.0,14.0,11.0,10.0,12.0,2.0,...,0.0,0.0,0.0,0.0,731.0,-1.0,0.0,NaN,SRFXAXNBRFQN30S,2021-12-17 13:25:57.623929
8,0.0,12.0,2.0,11.0,13.0,14.0,11.0,10.0,12.0,2.0,...,0.0,0.0,0.0,0.0,2787.0,1.0,0.0,NaN,SRFXAXNBRFQN30S,2021-12-17 13:25:57.623929
9,1.0,12.0,2.0,11.0,13.0,14.0,11.0,10.0,12.0,2.0,...,0.0,0.0,0.0,0.0,601.0,-1.0,0.0,NaN,SRFXAXNBRFQN30S,2021-12-17 13:25:57.623929


In [109]:
import torch

y = 10

for batch, x in enumerate(range(0, len(game_df), y)):
    v_headers = ['value']
    r_headers = ['reward']
    a_headers = ['action']
    s_headers = [h for h in game_df if 'state' in h]
    p_headers = [h for h in game_df if 'prob' in h]
    
    state = game_df[s_headers].iloc[x:x+y]

    s_target = game_df[s_headers].shift(periods = -1, axis = 0).iloc[x:x+y]
    if True in s_target.iloc[-1].isna().tolist():
        s_target.iloc[-1] = state.iloc[-1]
        s_target['state0'].iloc[-1] = 0 if s_target['state0'].iloc[-1] == 1 else 1
    s_target = torch.tensor(s_target.values)
    
    state = torch.tensor(state.values)
    
    p_target = torch.tensor(game_df[p_headers].iloc[x:x+y].values)
    v_target = torch.tensor(game_df[v_headers].iloc[x:x+y].values)
    r_target = torch.tensor(game_df[r_headers].iloc[x:x+y].values)

    a_target = game_df[a_headers].shift(axis = 0).iloc[x:x+y]
    if True in a_target.iloc[0].isna().tolist():
        a_target.iloc[0] = -1
    a_target = torch.tensor((a_target + 1).values)
    print(a_target)
    break

tensor([[   0.],
        [3308.],
        [ 667.],
        [3365.],
        [ 537.],
        [3957.],
        [  81.],
        [3757.],
        [ 732.],
        [2788.]], dtype=torch.float64)


In [61]:
print(game_df['value'].iloc[2:])

2      1.0
3     -1.0
4      1.0
5     -1.0
6      1.0
      ... 
116    1.0
117   -1.0
118    1.0
119   -1.0
120    1.0
Name: value, Length: 119, dtype: float64


In [59]:
import numpy as np

board = np.zeros((64))
s_headers = [h for h in game_df if 'state' in h]

for x in range(len(game_df[s_headers]) - 1):
    c = game_df[s_headers].iloc[x]
    n = game_df[s_headers].iloc[x + 1]
    
    #Show current board
    print(c.iloc[0])
    for i in range(len(c)-1):
        board[i] = c.iloc[ i + 1]
    print(board.reshape((8,8)))
    
    #Show next board
    print(n.iloc[0])
    for i in range(len(n)-1):
        board[i] = n.iloc[ i + 1]
    print(board.reshape((8,8)))
    
    #Find move
    mov = n-c
    print(mov[mov != 0])
    break

0.0
[[12. 10. 11. 13. 14. 11. 10. 12.]
 [ 9.  9.  9.  9.  9.  9.  9.  9.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.]
 [ 3.  3.  3.  3.  3.  3.  3.  3.]
 [ 6.  4.  5.  7.  8.  5.  4.  6.]]
1.0
[[12. 10. 11. 13. 14. 11. 10. 12.]
 [ 9.  9.  9.  9.  9.  9.  9.  9.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.]
 [ 3.  2.  2.  2.  2.  2.  2.  2.]
 [ 2.  3.  3.  3.  3.  3.  3.  3.]
 [ 6.  4.  5.  7.  8.  5.  4.  6.]]
state0     1.0
state41    1.0
state49   -1.0
dtype: float64


In [60]:
p_headers = [h for h in game_df if 'prob' in h]

actions = []
for i, row in game_df[p_headers].iterrows():
    print(row[(row > 0)].to_dict())
    break
    actions.append({k.replace('prob',''):v for k, v in row[(row > 0)].to_dict().items()})

print(pd.DataFrame(actions))

{'prob3104': 0.04, 'prob3112': 0.08, 'prob3169': 0.06, 'prob3177': 0.04, 'prob3234': 0.04, 'prob3242': 0.04, 'prob3299': 0.04, 'prob3307': 0.08, 'prob3364': 0.04, 'prob3372': 0.08, 'prob3429': 0.04, 'prob3437': 0.04, 'prob3494': 0.08, 'prob3502': 0.04, 'prob3559': 0.06, 'prob3567': 0.04, 'prob3688': 0.04, 'prob3690': 0.04, 'prob4013': 0.04, 'prob4015': 0.04}
Empty DataFrame
Columns: []
Index: []


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import pydot
from networkx.drawing.nx_pydot import graphviz_layout

G = nx.Graph()
for s in search.tree:
    for a in search.tree[s].n_s:
        G.add_edge(s,a)
        
pos = graphviz_layout(G, prog='dot')
#Display network graph -----------------------------
nx.draw(
    G, #Graph nodes & connections
    pos, #Position of graph
    with_labels=True #Labels on nodes
)
plt.rcParams['figure.figsize'] = [40, 40] #Resize graph
plt.show()